In [1]:
!pip install sentence_transformers
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver 

**Necessary Libraries**

In [2]:
import torch
import pickle
import numpy as np
import pandas as pd
from langchain.schema import Document
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-06-09 11:00:50.513047: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-09 11:00:50.513167: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-09 11:00:50.671998: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

**Reading the Data**

In [4]:
data = pd.read_json("/kaggle/input/drooid-work/news.article.json")
data

,articleBody,dateModified,scrapedDate,source,title
0,"Sanjay Raut, a member of the Shiv Sena (UBT) p...",{'$date': '2023-10-25T06:35:50.000Z'},{'$date': '2023-10-27T13:12:18.339Z'},https://www.thehansindia.com/,Shiv Sena MP Sanjay Raut Responds To 'Hamas' R...
1,"Kozhikode (Kerala) [India], October 27 (ANI): ...",NaN,{'$date': '2023-10-27T13:12:45.595Z'},https://www.aninews.in/,At IUML's pro-Palestine rally in Kerala Tharoo...
2,"Mumbai, Oct 24 (PTI) Maharashtra Chief Ministe...",{'$date': '2023-10-25T02:14:27.000Z'},{'$date': '2023-10-27T13:12:18.339Z'},https://thefederal.com/,Uddhav buried Bal Thackeray's 'Hindutva' for p...
3,"Sensex, Nifty rebound over 1 pc after six sess...",NaN,{'$date': '2023-10-27T13:12:41.618Z'},https://english.varthabharati.in/,"New Bills replacing IPC, CrPC, Evidence Act wi..."
4,"October 26, 2023 08:15 pm | Updated 08:38 pm I...",{'$date': '2023-10-26T14:45:24.000Z'},{'$date': '2023-10-27T13:12:45.595Z'},https://www.thehindu.com/,"Israel biggest terrorist nation in the world, ..."
...,...,...,...,...,...
37416,Lebanese media are reporting renewed IDF strik...,NaN,{'$date': '2024-03-31T03:10:17.646Z'},https://www.timesofisrael.com/,Lebanese media reports renewed IDF strikes in ...
37417,Amid escalating tensions and concerns over pot...,{'$date': '2024-03-31T03:03:52.000Z'},{'$date': '2024-03-31T03:09:38.331Z'},https://www.deccanchronicle.com/,"US approves additional bombs, warplanes sales ..."
37418,"Haaretz.com, the online English edition of Haa...",{'$date': '2024-03-28T00:00:00.000Z'},{'$date': '2024-03-31T03:09:46.683Z'},https://www.haaretz.com/,Israel Publishes Video of Islamic Jihad Terror...
37419,"The UN secretary general, António Guterres, ha...",{'$date': '2024-03-31T00:00:00.000Z'},{'$date': '2024-03-31T03:09:50.586Z'},https://www.theguardian.com/,United Nations secretary general condemns expl...


**Extracting the articles out of the json file**

In [5]:
text_corpus = []

for i in data['articleBody']:
    text_corpus.append(i)

**Spilting the article into smaller chunks to create the embedding vectors.**

In [6]:
documents = [Document(page_content=text) for text in text_corpus]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
all_splits = text_splitter.split_documents(documents)

In [7]:
all_splits[:5]

[Document(page_content='Sanjay Raut, a member of the Shiv Sena (UBT) party, responded to the Maharashtra chief minister\'s statement that Eknath Shinde "himself is Hamas" and that the Shiv Sena group led by Uddhav Thackeray is capable of collaborating with "Hamas and Lashkar-e-Taiba for their own selfishness" on Wednesday by claiming that Eknath Shinde is Hamas.\n\n\n\nRaut made fun of Shinde by claiming, "He himself is Hamas. Hamas and Lashkar-e-Taiba, two terrorist groups, are completely irrelevant in Maharashtra. But the BJP is to blame for sowing the worms in their (the Shinde faction\'s) thoughts, said Raut.\n\nWhen Shinde made a statement at the Tuesday Dussehra rally in Mumbai\'s Azad Maidan, Raut reacted to it. As part of the opposition alliance INDIA, Uddhav Thackeray\'s Shiv Sena (UBT) has formed an alliance with Congress and the Samajwadi Party. Shinde remarked of this alliance: "For their own selfishness, they will tie the knot with Hamas and Lashkar-e-Taiba."'),
 Document(

**Using sentence transformer to create the embedding of the chunks generated above.**

In [8]:
embed_model = SentenceTransformer("all-mpnet-base-v2").to(device)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
split_texts = [split.page_content for split in all_splits]
embeddings = embed_model.encode(split_texts, batch_size=64)

Batches:   0%|          | 0/2861 [00:00<?, ?it/s]

**Saving the Embedding vectors with their corresponding chunks.**

In [10]:
with open('documents.pkl', 'wb') as f:
    pickle.dump(all_splits, f)

np.save('embeddings.npy', embeddings)